# [LSA算法简单理解](https://www.cnblogs.com/sancallejon/p/4963630.html)


## 描述
- 本文主要目的是要未来再现LSA算法


## 1 构建词汇-文本矩阵

In [41]:
import re 
title_dict_raw={
 "c1": "Human machine interface for Lab ABC computer applications",
"c2": "A survey of user opinion of computer system response time",
"c3": "The EPS user interface management system",
"c4": "System and human system engineering testing of EPS",
"c5": "Relation of user-perceived response time to error measurement",
"m1": "The generation of random, binary, unordered trees",
"m2": "The intersection graph of paths in trees",
"m3": "Graph minors IV: Widths of trees and well-quasi-ordering",
"m4": "Graph minors: A survey",   
}

title_dict={ idx:[i.lower() for i in re.split(': | ',val) if len(i)>2] for idx,val in title_dict_raw.items()}
list_voc=['human','interface','computer','user','system','response','time','eps','survey','trees','graph','minors']


import pandas as pd
df_词汇文本矩阵=pd.DataFrame(index=list_voc,columns=title_dict.keys()).fillna(0)
df_词汇文本矩阵

for idx,val in title_dict.items():
    for word in list_voc:
        if word in val:
            df_词汇文本矩阵.loc[word,idx]=val.count(word)
display(df_词汇文本矩阵    )

,c1,c2,c3,c4,c5,m1,m2,m3,m4
human,1,0,0,1,0,0,0,0,0
interface,1,0,1,0,0,0,0,0,0
computer,1,1,0,0,0,0,0,0,0
user,0,1,1,0,0,0,0,0,0
system,0,1,1,2,0,0,0,0,0
response,0,1,0,0,1,0,0,0,0
time,0,1,0,0,1,0,0,0,0
eps,0,0,1,1,0,0,0,0,0
survey,0,1,0,0,0,0,0,0,1
trees,0,0,0,0,0,1,1,1,0


## 2 进行SVD分解 & 3 得到词汇与文本的二维坐标向量

In [176]:
from scipy.linalg import  svd
u, s, vt=svd(df_词汇文本矩阵.astype(float))
n_dim=3
df词汇=pd.DataFrame(u[:,:n_dim],index=list_voc)
df矩阵=pd.DataFrame(np.diag(s[:n_dim]))
df文章=pd.DataFrame(vt[:n_dim,:],columns=title_dict.keys())
display(df词汇)
display(df矩阵)
display(df文章)

,0,1,2
human,-0.249969,-0.083824,0.242852
interface,-0.211982,-0.058797,0.131777
computer,-0.244325,0.041638,-0.266867
user,-0.324026,0.014169,-0.178338
system,-0.691684,-0.122151,0.264117
response,-0.219249,0.083074,-0.481888
time,-0.219249,0.083074,-0.481888
eps,-0.329670,-0.111293,0.331381
survey,-0.204206,0.269615,-0.292803
trees,-0.014719,0.504020,0.225495


,0,1,2
0,3.26779,0.000000,0.000000
1,0.00000,2.539068,0.000000
2,0.00000,0.000000,2.232354


,c1,c2,c3,c4,c5,m1,m2,m3,m4
0,-0.216133,-0.582271,-0.476580,-0.600714,-0.134188,-0.004504,-0.016550,-0.027045,-0.085032
1,-0.039772,0.145494,-0.109517,-0.173063,0.065437,0.198506,0.449839,0.631395,0.539076
2,0.048273,-0.644014,0.245900,0.493858,-0.431731,0.101012,0.179823,0.222549,-0.009626


## 4 检索
- 提问式 human computer interaction

In [177]:
xq=np.array(xq).reshape(1,-1)
Dq=(np.dot(np.dot(xq,df词汇),df矩阵))

for i_s in df文章.columns:
    print(i_s,np.arccos(np.dot(df文章[i_s].values,Dq.flatten()))/np.pi*180)
    
# 夹角为90度附近说明关联性很差    

c1 69.46495838305033
c2 16.371599893184676
c3 39.794672955275225
c4 15.769164928532657
c5 76.53315923300427
m1 91.11172769658938
m2 91.78170957158632
m3 92.05599775998095
m4 85.40445493586631


In [178]:
pwd

'/Users/pengju/python-workspace/algorithm_playground/机器学习算法/LSA算法'